In [1]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd 
import numpy as np
from src.config import config, MODEL_CONFIG, PREP_CONFIG
from transformers import AutoTokenizer

In [3]:
# 1. Taking a look at cleaned dataset

In [4]:
stripped = pd.read_parquet(config.data.news.stripped)

In [5]:
stripped.columns

Index(['time', 'stocks', 'title', 'body', 'intra_day_time', 'channels',
       'company_name', 'short_name', 'author', 'parsed_body', 'cls_token',
       'staleness'],
      dtype='object')

In [6]:
stripped[["title", "stocks", "parsed_body"]].tail()

,title,stocks,parsed_body
index,,,
4578113,Cybin Announces Renewed At-The-Market Equity P...,CYBN,the company announces renewed atthemarket eq...
4578114,Environmental Icon Dr. Jane Goodall's Roots & ...,TM,environmental icon dr jane goodalls roots sho...
4578116,Chindata Group Unveils Pioneering Full-Stack S...,CD,chindata group unveils pioneering fullstack so...
4578117,Orbital Infrastructure Group Inc. Intends to E...,OIG,the company inc intends to enter into purcha...
4578118,"IBRX FINAL DEADLINE TUESDAY: ROSEN, TRUSTED IN...",IBRX,ibrx final deadline tuesday rosen trusted inve...


In [ ]:
# 2. Take a look at input_ids and if tokenizer works correctly

In [12]:
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)

In [13]:
ld = pd.read_parquet(config.data.learning_dataset)

In [14]:
#cleaned = ld = pd.read_parquet(config.data.merged) # requires more memory

In [3]:
tokenizer = AutoTokenizer.from_pretrained(PREP_CONFIG.tokenizer)

In [44]:
mask = ld.parsed_body.apply(lambda x: "today announced" in x)

In [47]:
ld = ld[mask]

In [60]:
n = np.random.randint(0, ld.shape[0])
index = ld.index[n]
print(index)
entry = title_inputs_ids.loc[index, :].values

print(ld.loc[index, "stocks"])
print(f"{tokenizer.decode(entry)} \n")
print(ld.loc[index, "parsed_body"])

for x in entry:
    print(f"{x} --- {tokenizer.decode(x)}")


3944556
BLK
[CLS] [UNK] canada announces october cash distributions for the [UNK] etfs [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

BlackRock® Canada Announces October Cash Distributions for the iShares® ETFs. the company Asset Management Canada Limited , an indirect, wholly owned subsidiary of the company , today announced the October 2020 cash distributions for the iShares ETFs listed on the TSX or NEO which pay on a monthly basis. Unitholders of record of a fund will receive cash distributions payable in respect of that fund. As a fiduciary to investors and a leading provider of financial technology, our clients turn to us for the solutions they need when planning for their most important goals. As , the firm managed approximately US$7.81 trillion in assets on behalf of investors worldwide. With more than twenty years of experience, a global line up of 900+ exchange traded funds (ETFs) and US$2.32 tri

# Average number of token in news

In [4]:
title_inputs_ids = pd.read_parquet(config.data.news.input_ids)


In [5]:
n_padding = title_inputs_ids[title_inputs_ids == 0].sum(axis=1)

# Predictions from Model

In [4]:
ld = pd.read_parquet(config.data.learning_dataset, columns=["z_score", "z_score_class", "parsed_body"])
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)
masks = pd.read_parquet(config.data.news.title_only.masks)

In [5]:
ld_mini = ld.iloc[-10000:,:]

In [6]:
index = ld.index[0]

In [7]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm


In [21]:
from transformers import AutoTokenizer, BertForSequenceClassification, BertTokenizer
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained("/gxfs_work/cau/sunms534/trading_bot/data/models/ProsusAI/finbert", num_labels=3, local_files_only=True)

prob = []
preds = []
actuals = []
with torch.no_grad():
    for index in tqdm(ld_mini.index):
        input_id = torch.tensor([title_inputs_ids.loc[index, :].tolist()], dtype=torch.int32)
        mask = torch.tensor([masks.loc[index, :].tolist()])
        result = finbert(input_id, mask)
        probs = F.softmax(result.logits).numpy()
        prob.append(probs.max())
        preds.append((probs.argmax()  + 1) % 2)
        actuals.append(ld.loc[index, "z_score_class"])
        #print(result)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

  0%|          | 0/10000 [00:00<?, ?it/s]

/scratch/SlurmTMP/sunms534.10715314/ipykernel_1460224/3251329861.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(result.logits).numpy()


In [44]:
F.softmax(result.logits).numpy().argmax()

/scratch/SlurmTMP/sunms534.10715314/ipykernel_1460224/3628110247.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(result.logits).numpy().argmax()


0

In [75]:
tokenizer.decode(title_inputs_ids.loc[ld_mini.index[532]])

'[CLS] robotexts increase while robocalls drop for third consecutive month, according to robokiller insights [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
len(preds)

10000

In [58]:
df = pd.DataFrame(zip(preds,actuals, prob))

In [59]:
df.loc[:, 2].describe()

count    10000.000000
mean         0.825836
std          0.124616
min          0.384929
25%          0.757534
50%          0.875642
75%          0.923297
max          0.969893
Name: 2, dtype: float64

In [60]:
df = df[df.iloc[:, 2] > 0.95]

In [61]:
len(df)

227

In [62]:
(df.iloc[:, 0] == df.iloc[:, 1]).sum()

115

In [63]:
df.iloc[:, 1].value_counts()

1
1    119
2     65
0     43
Name: count, dtype: int64

In [64]:
df.iloc[:, 0].value_counts()

0
1    210
0     17
Name: count, dtype: int64

In [70]:
df.iloc[:, 2].idxmax()

532

In [72]:
df.loc[532, :]

0    0.000000
1    1.000000
2    0.969893
Name: 532, dtype: float64

In [74]:
ld.loc[ld_mini.index[532], "parsed_body"]

'Robotexts Increase While Robocalls Drop For Third Consecutive Month, According to Robokiller Insights. Americans received 14.2 billion robotexts and 4.6 billion robocalls in May, a 9% increase and 10% decrease, respectively, according to Robokiller Insights. The two categories continue to follow diverging paths — this marks the third consecutive month over month increase in SMS spam and the second consecutive month over month decrease in unwanted calls. Government anti robocall efforts continue to pay dividends The decrease in robocalls can be attributed largely to the government\'s successful efforts to crack down on specific high volume robocalls. Yet, although the quantity of robocalls is dropping, the damage they cause is escalating. Americans have lost $29 billion to robocalls already in 2023, outpacing 2022\'s record losses. Scammers are getting better results despite fewer calls, signifying they continue to refine their tactics and double down on scams that work. According to t